In [2]:
import sys
sys.path.append(r'C:\Users\ICN_admin\Documents\icn\icn_bids')
sys.path.append(r'C:\Users\ICN_admin\Documents\icn')
import icn_bids
from bids import BIDSLayout
import mne_bids
import mne
import numpy as np
import os
import pybv

In [3]:
def preprocess_mov(mov_dat):
    """
    clean a TTL signal into continous movement block
    """
    # the TIME OFF in the TTL signal is ~50 ms
    MOV_ON = False
    mov_new = np.zeros(mov_dat.shape[0])
    mov_new[0] = mov_dat[0]
    mov_on_set = 0
    for i in range(mov_dat.shape[0]):
        if i > 0 and mov_dat[i] > 1:
            MOV_ON = True
            mov_on_set = i
        if (i - mov_on_set) > 100 and mov_dat[i] < 1:
            mov_new[i] = 0
            MOV_ON = False
        if MOV_ON is True:
            mov_new[i] = 1
    return mov_new

def set_chtypes(vhdr_raw):
    """
    define MNE RawArray channel types
    """
    #print('Setting new channel types...')
    remapping_dict = {}
    for ch_name in vhdr_raw.info['ch_names']:
        if ch_name.startswith('ECOG'):
            remapping_dict[ch_name] = 'ecog'
        elif ch_name.startswith(('LFP', 'STN')):
            remapping_dict[ch_name] = 'seeg'
        elif ch_name.startswith('EMG'):
            remapping_dict[ch_name] = 'emg'
        # mne_bids cannot handle both eeg and ieeg channel types in the same data
        elif ch_name.startswith('EEG'):
            remapping_dict[ch_name] = 'misc'
        elif ch_name.startswith(('MOV', 'ANALOG', 'ROT', 'ACC', 'AUX', 'X', 'Y', 'Z')):
            remapping_dict[ch_name] = 'misc'
        else:
            remapping_dict[ch_name] = 'misc'
    vhdr_raw.set_channel_types(remapping_dict, verbose=False)
    return vhdr_raw

In [5]:
# now setup as function

# I defined a copy of the BIDS Beijing dataset here
PATH_BEIJING = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here'

# and want to save the updated files in this directory 
PATH_UPDATE = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003'
# using pybids I read the run files 
layout = BIDSLayout(PATH_BEIJING)
for run_file in layout.get(extension='.vhdr'):
    print("writing "+str(run_file))
    #run_file = layout.get(extension='.vhdr')[0]

    # get the bids_path
    entities = mne_bids.get_entities_from_fname(run_file)
    bids_path = mne_bids.BIDSPath(subject=entities["subject"], session=entities["session"], task=entities["task"], \
        run=entities["run"], acquisition=entities["acquisition"], datatype="ieeg", root=PATH_BEIJING)

    # then read the brainvision file 
    raw_arr = mne.io.read_raw_brainvision(run_file)

    entities = mne_bids.get_entities_from_fname(run_file)

    # and the electrode.tsv file 
    elec_file = layout.get(extension='.tsv', suffix="electrodes", space="mni", subject=entities["subject"])[0].get_df()

    # first, add the fixed channel to the BIDS Data and create a new MNE RawArray using that 
    data_ = raw_arr.get_data()
    print(data_.shape)

    # then I add for every TTL channel the cleaned version 
    add_ = []
    add_label = []
    for ch_idx, ch in enumerate(raw_arr.ch_names):
        if "TTL" in ch:
            add_.append(preprocess_mov(data_[ch_idx, :]))
            add_label.append("TTL_"+str(len(add_))+"_clean")

    # stack the results to the previous brainvision data 
    data_new = np.concatenate((data_, np.vstack(add_)), axis=0)
    ch_l = raw_arr.ch_names.copy()
    [ch_l.append(l) for l in add_label] # updated channel list is now in ch_l 
    
    folder_dummy = r'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy'
    pybv.write_brainvision(data=data_new, \
                       sfreq=raw_arr.info["sfreq"], 
                       ch_names=ch_l,
                      fname_base='dummy', folder_out=folder_dummy)

    # I will now read this file using mne io again 
    raw_arr_read_ = mne.io.read_raw_brainvision(os.path.join(folder_dummy, 'dummy.vhdr'))

    # now I define the new electrode tsv, that will be hopefully written out by mne_bids
    elec = np.empty(shape=(len(raw_arr_read_.ch_names), 3))
    for ch_idx, ch in enumerate(raw_arr_read_.ch_names):
        try:
            elec[ch_idx, :] = elec_file[elec_file["name"] == ch][["x", "y", "z"]]
        except:
            elec[ch_idx, :] = ["NaN", "NaN", "NaN"]

    montage = mne.channels.make_dig_montage(ch_pos=dict(zip(raw_arr_read_.ch_names, elec)), \
                                            coord_frame='mni_tal')

    raw_arr_read_.set_montage(montage, on_missing='warn')

    # now adapt the channel types with Jonathan's function 
    raw_arr_read_ = set_chtypes(raw_arr_read_)

    raw_arr_read_.info['line_freq'] = 50 # otherwise we cannot write BIDS
    
    bids_path_update = bids_path
    bids_path_update.root = PATH_UPDATE

    mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)
    

writing <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr'>
Extracting parameters from <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr'>...
Setting channel info structure...
(26, 1312098)
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy\dummy.vhdr...
Setting channel info structure...
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy\dummy.vhdr...
Setting channel info structure...

Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\participants.tsv'...

participant_id	age	sex	hand
sub-FOG006	n/a	n/a	n/a
sub-FOG008	n/a	n/a	n/a
sub-FOG010	n/a	

<ipython-input-5-c48995d55bba>:64: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr_read_.set_montage(montage, on_missing='warn')
<ipython-input-5-c48995d55bba>:74: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events_data and event_id parameters. If this is resting state data it is recommended to name the task "rest".
  mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)



Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\sub-FOG006_ses-EphysMedOn_scans.tsv'...

filename	acq_time
ieeg/sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr	n/a
ieeg/sub-FOG006_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr	n/a
Wrote C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\sub-FOG006_ses-EphysMedOn_scans.tsv entry with ieeg\sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr.
writing <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr'>
Extracting parameters from <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr'>...
S

<ipython-input-5-c48995d55bba>:64: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr_read_.set_montage(montage, on_missing='warn')
<ipython-input-5-c48995d55bba>:74: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events_data and event_id parameters. If this is resting state data it is recommended to name the task "rest".
  mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)



Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\sub-FOG006_ses-EphysMedOn_scans.tsv'...

filename	acq_time
ieeg/sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr	n/a
ieeg/sub-FOG006_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr	n/a
Wrote C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\sub-FOG006_ses-EphysMedOn_scans.tsv entry with ieeg\sub-FOG006_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr.
writing <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr'>
Extracting parameters from <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr

...

<ipython-input-5-c48995d55bba>:64: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr_read_.set_montage(montage, on_missing='warn')
<ipython-input-5-c48995d55bba>:74: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events_data and event_id parameters. If this is resting state data it is recommended to name the task "rest".
  mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)


(25, 1276661)
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy\dummy.vhdr...
Setting channel info structure...
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\folder_dummy\dummy.vhdr...
Setting channel info structure...

Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\participants.tsv'...

participant_id	age	sex	hand
sub-FOG006	n/a	n/a	n/a
sub-FOG008	n/a	n/a	n/a
sub-FOG010	n/a	n/a	n/a
sub-FOG011	n/a	n/a	n/a
sub-FOG013	n/a	n/a	n/a

Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\participants.json'...

{
    "participant_id": {
        "Description": "Unique participant identifier"
    },
    "age": {
        "Description": "Age of the participant at time of testing",
        "Units": "years"
    },
    "sex": {
        "Description": "Biological sex of the participant",
        "Levels": {
          

<ipython-input-5-c48995d55bba>:64: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr_read_.set_montage(montage, on_missing='warn')
<ipython-input-5-c48995d55bba>:74: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events_data and event_id parameters. If this is resting state data it is recommended to name the task "rest".
  mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)



Writing 'C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\ses-EphysMedOff\sub-FOGC001_ses-EphysMedOff_scans.tsv'...

filename	acq_time
ieeg/sub-FOGC001_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr	n/a
ieeg/sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_ieeg.vhdr	n/a
Wrote C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\ses-EphysMedOff\sub-FOGC001_ses-EphysMedOff_scans.tsv entry with ieeg\sub-FOGC001_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr.
writing <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOGC001\ses-EphysMedOff\ieeg\sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_ieeg.vhdr'>
Extracting parameters from <BIDSFile filename='C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_here\sub-FOGC001\ses-EphysMedOff\ieeg\sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_r

<ipython-input-5-c48995d55bba>:64: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr_read_.set_montage(montage, on_missing='warn')
<ipython-input-5-c48995d55bba>:74: RuntimeWarning: No events found or provided. Please add annotations to the raw data, or provide the events_data and event_id parameters. If this is resting state data it is recommended to name the task "rest".
  mne_bids.write_raw_bids(raw=raw_arr_read_, bids_path=bids_path_update, overwrite=True)


In [9]:
for run_file in layout.get(extension='.vhdr'):
    # read
    mne_bids.read_raw_bids(bids_path)
    entities = mne_bids.get_entities_from_fname(run_file)
    bids_path = mne_bids.BIDSPath(subject=entities["subject"], session=entities["session"], task=entities["task"], \
        run=entities["run"], acquisition=entities["acquisition"], datatype="ieeg", root=PATH_UPDATE)
    raw_arr = mne_bids.read_raw_bids(bids_path)

Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\ses-EphysMedOff\ieeg\sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\ses-EphysMedOff\ieeg\sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG006\ses-EphysMedOn\ieeg\sub-FOG006_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\w

<ipython-input-9-c906679b1a67>:3: RuntimeWarning: Did not find any events.tsv associated with sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01.

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\**\sub-FOGC001_ses-EphysMedOff*events.tsv"
  mne_bids.read_raw_bids(bids_path)
<ipython-input-9-c906679b1a67>:3: RuntimeWarning: Expected to find a single coordsystem file associated with sub-FOGC001_ses-EphysMedOff_task-Rest_acq-StimOff_run-01, but found 2: "['C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOGC001\\ses-EphysMedOff\\ieeg\\sub-FOGC001_ses-EphysMedOff_acq-StimOff_space-mni_coordsystem.json', 'C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOGC001\\ses-EphysMedOff\\ieeg\\sub-FOGC001_ses-EphysMedOff_acq-StimOff_space-mri_coordsystem.json']".

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beij

Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG008\ses-EphysMedOn\ieeg\sub-FOG008_ses-EphysMedOn_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_aga

<ipython-input-9-c906679b1a67>:3: RuntimeWarning: Expected to find a single coordsystem file associated with sub-FOG008_ses-EphysMedOn_task-ButtonPress_acq-StimOff_run-01, but found 2: "['C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG008\\ses-EphysMedOn\\ieeg\\sub-FOG008_ses-EphysMedOn_acq-StimOff_space-mni_coordsystem.json', 'C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG008\\ses-EphysMedOn\\ieeg\\sub-FOG008_ses-EphysMedOn_acq-StimOff_space-mri_coordsystem.json']".

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG008\**\sub-FOG008_ses-EphysMedOn*coordsystem.json"
  mne_bids.read_raw_bids(bids_path)
<ipython-input-9-c906679b1a67>:7: RuntimeWarning: Did not find any events.tsv associated with sub-FOG008_ses-EphysMedOn_task-Rest_acq-StimOff_run-01.

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijin

Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG011\ses-EphysMedOff\ieeg\sub-FOG011_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG011\ses-EphysMedOff\ieeg\sub-FOG011_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG011\ses-EphysMedOff\ieeg\sub-FOG011_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG011\ses-EphysMedOff\ieeg\sub-FOG011_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Too

<ipython-input-9-c906679b1a67>:7: RuntimeWarning: Expected to find a single coordsystem file associated with sub-FOG011_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01, but found 2: "['C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG011\\ses-EphysMedOff\\ieeg\\sub-FOG011_ses-EphysMedOff_acq-StimOff_space-mni_coordsystem.json', 'C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG011\\ses-EphysMedOff\\ieeg\\sub-FOG011_ses-EphysMedOff_acq-StimOff_space-mri_coordsystem.json']".

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG011\**\sub-FOG011_ses-EphysMedOff*coordsystem.json"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-9-c906679b1a67>:3: RuntimeWarning: Did not find any events.tsv associated with sub-FOG011_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01.

The search_str was "C:\Users\ICN_admin\Documents\Decod

Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG013\ses-EphysMedOff\ieeg\sub-FOG013_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG013\ses-EphysMedOff\ieeg\sub-FOG013_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG013\ses-EphysMedOff\ieeg\sub-FOG013_ses-EphysMedOff_task-Rest_acq-StimOff_run-01_channels.tsv.
Extracting parameters from C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOGC001\ses-EphysMedOff\ieeg\sub-FOGC001_ses-EphysMedOff_task-ButtonPress_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\ICN_admin\

<ipython-input-9-c906679b1a67>:7: RuntimeWarning: Did not find any events.tsv associated with sub-FOG013_ses-EphysMedOff_task-Rest_acq-StimOff_run-01.

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Beijing_again\BIDS_update_1003\sub-FOG013\**\sub-FOG013_ses-EphysMedOff*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-9-c906679b1a67>:7: RuntimeWarning: Expected to find a single coordsystem file associated with sub-FOG013_ses-EphysMedOff_task-Rest_acq-StimOff_run-01, but found 2: "['C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG013\\ses-EphysMedOff\\ieeg\\sub-FOG013_ses-EphysMedOff_acq-StimOff_space-mni_coordsystem.json', 'C:\\Users\\ICN_admin\\Documents\\Decoding_Toolbox\\write_Beijing_again\\BIDS_update_1003\\sub-FOG013\\ses-EphysMedOff\\ieeg\\sub-FOG013_ses-EphysMedOff_acq-StimOff_space-mri_coordsystem.json']".

The search_str was "C:\Users\ICN_admin\Documents\Decoding_Toolbox\write_Be

In [14]:
raw_arr.get_montage()

In [20]:
# but pybids can be used for that 
layout.get(subject="FOG006", extension='.tsv', suffix="electrodes")[0].get_df()

,name,x,y,z,size,material,manufacturer,group,hemisphere,type,impedance
0,ECOG_1_R_SM_HH,26.104,37.044,24.285,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
1,ECOG_2_R_SM_HH,39.523,33.709,39.122,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
2,ECOG_3_R_SM_HH,37.136,26.786,44.678,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
3,ECOG_4_R_SM_HH,25.900,3.755,56.742,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
4,ECOG_5_R_SM_HH,24.673,-4.449,56.121,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
5,ECOG_6_R_SM_HH,26.696,-14.655,56.537,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
6,ECOG_7_R_SM_HH,23.271,-32.886,50.249,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
7,ECOG_8_R_SM_HH,16.241,-40.031,50.028,3mm diameters,steel,"Huake Hengsheng, Beijing",RS,Right,ECOG,NaN
8,LFP_1_L_STN_PI,NaN,NaN,NaN,NaN,NaN,PINS,LD,Left,LFP,NaN
9,LFP_2_L_STN_PI,NaN,NaN,NaN,NaN,NaN,PINS,LD,Left,LFP,NaN
